# A Pythonic Object

Your application class probably don't need and should not implement as many special methods.
- But if you write a lib or a framework, the programmers who will use them may expect them to behave like the classes that python provides.
- Fulfilling that expectation is one way of being "Pythonic".


# Custom Vector Class

In order to demonstrate the many methods used to generate object representations, we will implement a 2 dimensional Vector class: *Vector2d* 

In [61]:
import math

class Vector2d:

    def __init__(self, x, y):   #constructor
        self.x = float(x)       # converting x an y to floats catches errors early
        self.y = float(y)

    def __repr__(self):        # string representation in console 
        class_name = type(self).__name__
        return f"{class_name}({self.x} {self.y})"

    def __str__(self):
        return str((self.x, self.y))

    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        rad = math.atan2(self.y, self.x)
        return math.degrees(rad)

In [62]:
v1 = Vector2d(1,2)
print(v1)   #string representation; it's what you get when you print an object
v2 = Vector2d(4,3)
print(v2)
v1 == v2

abs(v2)
bool(v2)

v3 = Vector2d(0,0)
bool(v3)

v4 = Vector2d(-1,0)
v4.angle()


(1.0, 2.0)
(4.0, 3.0)


180.0

## classmethod Versus staticmethod

- classmethod decorator changes the way the method is called, so it receives the class itself as the first argument, instead of an instance. Its most common use is for alternative constructors
- in contrast, the staticmethod decorator changes a method so that it receives no special first argument.
- in essence, a static method is just like a plain function that happens to live in a a class body, instead of being defined at the module level

In [49]:
class Demo:
    @classmethod
    def klassmeth(args):
        return args
    
    @staticmethod #good usecases are rare
    def statmeth(args):
        return args

print(Demo.klassmeth())
print(Demo.statmeth('bla'))

<class '__main__.Demo'>
bla


## A Hashable Vector2d

As defined, so far our Vector2d instances are unhashable:

In [63]:
v1 = Vector2d(1,2)
hash(v1)

TypeError: unhashable type: 'Vector2d'

- special method __ hash__
- Vector instances must be immutable

In [74]:
class Vector2d:

    def __init__(self, x, y):   
        self.__x = float(x)       # underscores make attribute private
        self.__y = float(y)

    @property  #marks the getter method of a property
    def x(self): # named after the public property it exposes: x
        return self.__x

    @property  #marks the getter method of a property
    def y(self): # named after the public property it exposes: x
        return self.__y

    def __repr__(self):        # string representation in console 
        class_name = type(self).__name__
        return f"{class_name}({self.x} {self.y})"

    def __str__(self):
        return str((self.x, self.y))

    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        rad = math.atan2(self.y, self.x)
        return math.degrees(rad)

    def __hash__(self):
        return hash((self.x, self.y))

In [75]:
v1 = Vector2d(1,1)
v2 = Vector2d(1.2,1)
# v1.__x = 2
hash(v1)


8389048192121911274

In [128]:
class MYProperty:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget

    def __get__(self, obj, objtype=None):
        # if obj is None:
        #     return self
        # if self.fget is None:
        #     raise AttributeError(f"property '{self._name}' has no getter")
        # return self.fget(obj)
        return 'bla'


class Test:

    def __init__(self, x, y):
        self.__x = x
        self.__y = y

    # @MYProperty
    def __get__(self, instance, owner):
        return self.__x

    # def x(self):
    #     return self.__x

t1 = Test(1,2) 
t1.x


AttributeError: 'Test' object has no attribute 'x'

You should only implement these special methods if your application needs them. End users don't care if the objects that make up an application are "Pythonic" or not.

## Private and "Protected" Attributes in Python

Suppose you have a Dog class that uses a *mood* instance attribute internally, without exposing it.
If you subclass Dog as Beagle and create your own *mood* instance attributey, you will clobber (overwrite) the *mood*
attribute used by the methods inherited from Dog.

To prevent this you add 2 leadding underscores. Python stores the name in __ dict__: *__ mood* 
becomes *_dog__mood*

This language feature goes by the name *name mangling*.



In [88]:
class Dog:

    def __init__(self):
        self.__mood = 'good'

chiwauwau = Dog()

chiwauwau.__dict__

{'mood': 'good'}

In [89]:
class Beagle(Dog):

    def __init__(self):
        super().__init__()
        self.__mood ='Bla'
        
frank = Beagle()
frank.__dict__

{'mood': 'Bla'}

- Be aware *name mangling* is about safety, not security: it's designed to prevent accidental access and not malicous prying.


The single underscore prefix has no special meaning to the Python interpreter when used in attributes names, but it's a very strong convention among Python programmers that you should not access such attribues from outside the class.

- single _prefixed attributes are called "protected" some even call it private.

# Vector2d to N-dim Vector

In [201]:
# Vector2d to N-dim Vector

import reprlib

class Vector:

    def __init__(self, components):   
        self._components = components

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):        # string representation in console 
        components = reprlib.repr(self._components[:7]) # returns string in form of '[0,1,2,3,4,5, ...]
        components = components[1:-1] + ', ' + str(self._components[7:])[1:-1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        return self._components[index]

## A slice-Aware __getitem__

Slice support can be achieved by __ getitem__

In [202]:
v1 = Vector([*range(8)])
len(v1)
type(v1[2:6])

list

- but the return value is of type list
- It would be better if a slice of a vector was also a Vector instance and not a list.

But lets firat look on the built-in slice object:

In [203]:
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()
print(s[1])
print(s[1:4]) #slice object
print(s[1:4:2])
print(s[1:4:2, 9]) #Surprise: the presence of a comma returns a tuple of 
print(s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


- slice has a method called *indices* which exposes the logic that's implemented to gracefully handle missing or negative indices and slices that are longer than the original sequence.

In [204]:
slice(None, 10, 2).indices(5)   # 'ABCDE'[:10:2] is the same as 'ABCDE'[0:5:2]

(0, 5, 2)

In [205]:
slice(-3, None, None).indices(5)  # 'ABCDE'[-3:] is the same as 'ABCDE'[2:5:1]

(2, 5, 1)

In [206]:
class Vector:

    def __init__(self, components):   
        self._components = components

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):        # string representation in console 
        components = reprlib.repr(self._components[:7]) # returns string in form of '[0,1,2,3,4,5, ...]
        components = components[1:-1] + ', ' + str(self._components[7:])[1:-1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        print(key)
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        else:
            return self._components[key]

In [199]:
v1 = Vector([*range(7)])
print(v1[-1])
print(v1[1:4])
print(v1[-1:])
print(v1[1,2]) # Vector does not support multidimensional indexing

-1
6
slice(1, 4, None)
(1, 2, 3)
slice(-1, None, None)
(6,)
(1, 2)


TypeError: list indices must be integers or slices, not tuple

# Hashing

Now we may be dealing with thousand of components, so building a tuple may be too costly.
Instead, I will apply the ^ (xor) to the hashes of every components in succession: v[0] ^ v[1] ^ v[2]

In [221]:
import functools
import operator
class Vector:

    def __init__(self, components):   
        self._components = components

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):        # string representation in console 
        components = reprlib.repr(self._components[:7]) # returns string in form of '[0,1,2,3,4,5, ...]
        components = components[1:-1] + ', ' + str(self._components[7:])[1:-1]
        return f"Vector({components})"

    def __str__(self):
        return str(tuple(self))

    def __eq__(self, other):
        # return tuple(self) == tuple(other)
        if len(self) != len(other):
            return False
        for a,b in zip(self, other):
            if a != b:
                return False
        return True

    def __hash__(self):
        hashes = map(hash, self._components)
        # return functools.reduce(lambda acc, cur: acc ^ cur, hashes, 0)
        return functools.reduce(operator.xor, hashes, 0)
    
    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        print(key)
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        else:
            return self._components[key]

    

In [222]:
v1 = Vector([*range(9)])
v2 = Vector([*range(9)])
print(v1 == v2)
print(hash(v1))
print(hash(v2))

True
8
8


# Inheritance

- super()
- 